## Purpose
* Merge the different patient data to create the final dataframe. lab_vital_merged_time.csv contains the filtered data (negative class sampled days 2-5, negative patients that underwent dialysis). Use this table to filter the patient info and past history.
* Get rid of columns with most missing values in the positive class.
* Get rid of rows with most missing values.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pat_adm = pd.read_csv('../data/intermediate/pat_admit_diag_cleaned.csv')
lab_vit = pd.read_csv('../data/intermediate/lab_vital_merged_time.csv')
past_hist = pd.read_csv('../data/intermediate/pasthist_processed.csv')
print(len(pat_adm))
print(len(lab_vit))
print(len(past_hist))

90804
51079
90804


In [21]:
pat_adm.set_index('patientunitstayid', inplace=True)
past_hist.set_index('patientunitstayid', inplace=True)
lab_vit.set_index('patientunitstayid', inplace=True)

In [23]:
combined_df = lab_vit.merge(pat_adm, right_index=True, left_index=True)
combined_df = combined_df.merge(past_hist, right_index=True, left_index=True)
len(combined_df)

51079

In [29]:
pos_mis_val = combined_df[combined_df['aki_label']==1].isna().sum()
num_pos_class = len(combined_df[combined_df['aki_label']==1])
print(num_pos_class)
pos_mis_val_fract = pos_mis_val/num_pos_class
cols_to_drop = pos_mis_val_fract[pos_mis_val_fract>0.1]
cols_to_drop

2295


min_result_FiO2                 0.273203
min_result_HCO3                 0.299346
min_result_MCH                  0.129847
min_result_MPV                  0.426144
min_result_RDW                  0.128976
min_result_anion gap            0.195207
min_result_bedside glucose      0.215686
min_result_magnesium            0.209586
min_result_pH                   0.285403
min_result_paCO2                0.296296
min_result_paO2                 0.285403
max_result_FiO2                 0.273203
max_result_HCO3                 0.299346
max_result_MCH                  0.129847
max_result_MPV                  0.426144
max_result_RDW                  0.128976
max_result_anion gap            0.195207
max_result_bedside glucose      0.215686
max_result_magnesium            0.209586
max_result_pH                   0.285403
max_result_paCO2                0.296296
max_result_paO2                 0.285403
delta_result_FiO2               0.273203
delta_result_HCO3               0.299346
delta_result_MCH

In [31]:
print(combined_df.shape)
combined_df.drop(columns=cols_to_drop.index.tolist(), inplace=True)
print(combined_df.shape)

(51079, 333)
(51079, 300)


In [56]:
row_null_sum_neg = combined_df.loc[combined_df['aki_label']==0].isnull().sum(axis=1).sort_values(ascending=True)
neg_to_drop = row_null_sum_neg[row_null_sum_neg>6]
len(neg_to_drop)

3384

In [55]:
row_null_sum_pos = combined_df.loc[combined_df['aki_label']==1].isnull().sum(axis=1).sort_values(ascending=True)
pos_to_drop = row_null_sum_pos[row_null_sum_pos>6]
len(pos_to_drop)

165

In [59]:
total_rows_todrop = neg_to_drop.index.tolist() + pos_to_drop.index.tolist()
#combined_df
len(total_rows_todrop)

3549

In [60]:
print(len(combined_df))
combined_df = combined_df[~combined_df.index.isin(total_rows_todrop)]
print(len(combined_df))

51079
47530


In [68]:
from sklearn.model_selection import train_test_split
# generate unique pateint set for splitting
patient_set = combined_df.groupby('patienthealthsystemstayid')['aki_label'].sum()
# if a pateint had more than one episode of aki, set it to 1
patient_set.loc[patient_set > 1]=1
train_val, test = train_test_split(patient_set, test_size=0.2)
train, val = train_test_split(train_val, test_size=0.25)
print(sum(train)/len(train))
print(sum(val)/len(val))
print(sum(test)/len(test))

0.046279958829583884
0.046537273930304365
0.04929422143802382


In [72]:
print(len(train))
print(len(val))
print(len(test))

27204
9068
9068


In [73]:
train_df = combined_df[combined_df['patienthealthsystemstayid'].isin(train.index.tolist())]
val_df = combined_df[combined_df['patienthealthsystemstayid'].isin(val.index.tolist())]
test_df = combined_df[combined_df['patienthealthsystemstayid'].isin(test.index.tolist())]
print(len(train_df))
print(len(val_df))
print(len(test_df))

28543
9488
9499


In [75]:
combined_df.columns.tolist()

['min_sao2',
 'max_sao2',
 'mean_sao2',
 'min_heartrate',
 'max_heartrate',
 'mean_heartrate',
 'min_respiration',
 'max_respiration',
 'mean_respiration',
 'days_of_data',
 'min_result_BUN',
 'min_result_Hct',
 'min_result_Hgb',
 'min_result_MCHC',
 'min_result_MCV',
 'min_result_RBC',
 'min_result_WBC x 1000',
 'min_result_bicarbonate',
 'min_result_calcium',
 'min_result_chloride',
 'min_result_creatinine',
 'min_result_glucose',
 'min_result_platelets x 1000',
 'min_result_potassium',
 'min_result_sodium',
 'max_result_BUN',
 'max_result_Hct',
 'max_result_Hgb',
 'max_result_MCHC',
 'max_result_MCV',
 'max_result_RBC',
 'max_result_WBC x 1000',
 'max_result_bicarbonate',
 'max_result_calcium',
 'max_result_chloride',
 'max_result_creatinine',
 'max_result_glucose',
 'max_result_platelets x 1000',
 'max_result_potassium',
 'max_result_sodium',
 'delta_result_BUN',
 'delta_result_Hct',
 'delta_result_Hgb',
 'delta_result_MCHC',
 'delta_result_MCV',
 'delta_result_RBC',
 'delta_result

In [ ]:
not_predictors = ['days_of_data', 'unitdischargestatus_Alive', 'patienthealthsystemstayid',\
                  'unitdischargeoffset', 'aki_offset']
train_df.drop(columns=not_predictors).to_csv('../data/intermediate/06162010_train.csv')
val_df.drop(columns=not_predictors).to_csv('../data/intermediate/06162010_val.csv')
test_df.drop(columns=not_predictors).to_csv('../data/intermediate/06162010_test.csv')